In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder \
.config("spark.hadoop.fs.s3a.access.key", os.environ["AWS_ACCESS_KEY"]) \
.config("spark.hadoop.fs.s3a.secret.key", os.environ["AWS_SECRET_KEY"]) \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/21 17:47:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [5]:
df.show(truncate=False)

+---+---+-------+----------+-------------------+
|a  |b  |c      |d         |e                  |
+---+---+-------+----------+-------------------+
|1  |2.0|string1|2000-01-01|2000-01-01 12:00:00|
|1  |2.0|string1|2000-01-01|2000-01-01 12:00:00|
|2  |3.0|string2|2000-02-01|2000-01-02 12:00:00|
|4  |5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [6]:
# df.groupBy('a').count().show(truncate=False)
df.distinct().show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [7]:
df.filter(df.a==1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [8]:
p_df = spark.read.parquet("/Users/morgan/Downloads/*.parquet")

In [9]:
p_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [12]:
p_df = spark.read.parquet("s3a://skku-sw-bigdata/data/*.parquet")

24/07/21 17:47:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


# 

In [14]:
p_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [19]:
p_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [23]:
p_df.groupBy("VendorID").count().orderBy(desc("count")).show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       2|2234632|
|       1| 729732|
|       6|    260|
+--------+-------+



In [24]:
p_df.select(avg("trip_distance")).show()

+------------------+
|avg(trip_distance)|
+------------------+
|3.6521691789583146|
+------------------+



In [25]:
p_df.select(min("tpep_pickup_datetime"), max("tpep_pickup_datetime")).show()

+-------------------------+-------------------------+
|min(tpep_pickup_datetime)|max(tpep_pickup_datetime)|
+-------------------------+-------------------------+
|      2002-12-31 22:59:39|      2024-02-01 00:01:15|
+-------------------------+-------------------------+



In [34]:
p_df.filter(p_df.tpep_pickup_datetime >= "2024-01-01").withColumn("pickup_dt", split("tpep_pickup_datetime", ' ')[0]).groupBy("pickup_dt").count().orderBy("pickup_dt").show(31)

+----------+------+
| pickup_dt| count|
+----------+------+
|2024-01-01| 81013|
|2024-01-02| 75519|
|2024-01-03| 82427|
|2024-01-04|102901|
|2024-01-05|103178|
|2024-01-06| 97117|
|2024-01-07| 67543|
|2024-01-08| 80034|
|2024-01-09| 93962|
|2024-01-10| 95000|
|2024-01-11|105010|
|2024-01-12|103655|
|2024-01-13|104758|
|2024-01-14| 94420|
|2024-01-15| 77033|
|2024-01-16| 93057|
|2024-01-17|110365|
|2024-01-18|110358|
|2024-01-19| 95951|
|2024-01-20|108768|
|2024-01-21| 84502|
|2024-01-22| 85650|
|2024-01-23| 99891|
|2024-01-24|105120|
|2024-01-25|110318|
|2024-01-26|105804|
|2024-01-27|110515|
|2024-01-28| 92837|
|2024-01-29| 84547|
|2024-01-30|101233|
|2024-01-31|102120|
+----------+------+
only showing top 31 rows



In [36]:
%%bash
ncal 2024

                                  2024
    January           February          March             April             
Mo  1  8 15 22 29        5 12 19 26        4 11 18 25     1  8 15 22 29   
Tu  2  9 16 23 30        6 13 20 27        5 12 19 26     2  9 16 23 30   
We  3 10 17 24 31        7 14 21 28        6 13 20 27     3 10 17 24      
Th  4 11 18 25        1  8 15 22 29        7 14 21 28     4 11 18 25      
Fr  5 12 19 26        2  9 16 23        1  8 15 22 29     5 12 19 26      
Sa  6 13 20 27        3 10 17 24        2  9 16 23 30     6 13 20 27      
Su  7 14 21 28        4 11 18 25        3 10 17 24 31     7 14 21 28      

    May               June              July              August            
Mo     6 13 20 27        3 10 17 24     1  8 15 22 29        5 12 19 26   
Tu     7 14 21 28        4 11 18 25     2  9 16 23 30        6 13 20 27   
We  1  8 15 22 29        5 12 19 26     3 10 17 24 31        7 14 21 28   
Th  2  9 16 23 30        6 13 20 27     4 11 18 25      